In [47]:
from deeplate.MMdata import MMData 
import os, stat
import subprocess
import numpy as np

In [63]:
#folder containing data
#folder = '/scicore/home/nimwegen/urchuegu/projects/prj_noise_across_conditions_main/cell_size/data/20180506/20180506_synrich_M02_1/'
#folder = '/scicore/home/nimwegen/urchuegu/projects/prj_noise_across_conditions_main/cell_size/data/20180315/20180315_m9gluc_M02_1/'
folder = '/scicore/home/nimwegen/urchuegu/projects/prj_noise_across_conditions_main/cell_size/data/20170427/20170424_highmed_1/'
#folder where to save analysis
folder_save = '/scicore/home/nimwegen/witzg/DeepPlateAnalysis'
#base folder name (e.g. "Analysis" in "Analysis_20180315_styph24h_M02_3")
basename = 'Analysis4'
#folder containing weights
weights_folder  = '/scicore/home/nimwegen/GROUP/MM_Data/DeepLearning/96Plates/watershed/'


In [64]:
MMobj = MMData(folder = folder)
positions, well = MMobj.get_position_names()
print('Number of positions: '+str(len(positions)))

Number of positions: 540


In [65]:
pos_seq = None
#Reeduce number of analyzed positions
#pos_seq = np.arange(269,270)

In [66]:
MMobj = MMData(folder = folder)
print(MMobj.channels)

['DIA (Dual)', 'GFP (Dual noPh3)']


In [67]:
#based on the above outpout set fluo and bright field channels
fluo_ch = 1
bf_ch = 0

In [68]:
folder_analyzed = folder_save+'/'+basename+'_'+os.path.basename(os.path.normpath(folder))+'/'
folder_logs = folder_save+'/'+basename+'_'+os.path.basename(os.path.normpath(folder))+'/logs/'
if not os.path.exists(folder_analyzed):
    os.makedirs(folder_logs)
if not os.path.exists(folder_logs):
    os.makedirs(folder_logs)

In [69]:
#run this for tests
#%%bash -s "$folder" "$weights_folder" "$folder_analyzed"
#python deep_plate_analysis_multiD.py $1 $2 $3 240 1 0 1
    

In [70]:
basename = os.path.basename(os.path.normpath(folder))
num_in_group = 60
if pos_seq is None:
    pos_seq = np.arange(len(positions))
partitioned = [pos_seq[i:i+num_in_group] for i  in range(0, len(pos_seq), num_in_group)]
for p in partitioned:
    filename = folder_analyzed+basename+'_'+str(p[0])+'_'+str(p[-1])+'.sh'

    f = open(filename, "w")
    
    #regular CPU
    '''f.write('#!/bin/bash\n')
    f.write('#SBATCH --job-name=test_JOB2 \n')
    f.write('#SBATCH --time=01:00:00 \n')
    f.write('#SBATCH --mem=8G \n')
    f.write('#SBATCH --qos=6hours \n')
    f.write('#SBATCH -e '+folder_logs+'error_'+basename+'_'+str(p[0])+'_'+str(p[-1])+'.log \n')
    f.write('#SBATCH -o '+folder_logs+'output_'+basename+'_'+str(p[0])+'_'+str(p[-1])+'.log \n')
    f.write('module load Python/3.5.2-goolf-1.7.20 \n')
    f.write('source /scicore/home/nimwegen/witzg/DeepPlateSegmenter/venv-deeplate/bin/activate \n')
    f.write('python deep_plate_analysis.py "'+
            folder+'" "'+weights_folder+'" "'+folder_analyzed+'" '+str(p[0])+' '+str(num_in_group)+' '+str(bf_ch)+' '+str(fluo_ch)+' \n')
    f.close()'''
    
    #GPU
    f.write('#!/bin/bash\n')
    f.write('#SBATCH --job-name=coliplate-gpu \n')
    f.write('#SBATCH --time=06:00:00 \n')
    f.write('#SBATCH --mem-per-cpu=32G \n')
    f.write('#SBATCH --ntasks=1 \n')
    f.write('#SBATCH --cpus-per-task=1 \n')
    #f.write('#SBATCH --qos=6hours \n')
    #f.write('#SBATCH --partition=k80 \n')
    f.write('#SBATCH --qos=emgpu \n')
    f.write('#SBATCH --partition=titanx \n')
    f.write('#SBATCH --gres=gpu:1 \n')
    
    #until mid-May 2018. Can be deleted then
    #f.write('#SBATCH --reservation=hpc-course \n')
    
    f.write('#SBATCH -e '+folder_logs+'error_'+basename+'_'+str(p[0])+'_'+str(p[-1])+'.log \n')
    f.write('#SBATCH -o '+folder_logs+'output_'+basename+'_'+str(p[0])+'_'+str(p[-1])+'.log \n')
    f.write('module load Python/3.5.2-goolf-1.7.20 \n')
    f.write('module load cuDNN \n')
    f.write('source /scicore/home/nimwegen/witzg/DeepPlateSegmenter/venv-deeplate-gpu/bin/activate \n')
    f.write('python deep_plate_analysis_multiD.py "'+
            folder+'" "'+weights_folder+'" "'+folder_analyzed+'" '+str(p[0])+' '+str(num_in_group)+' '+str(bf_ch)+' '+str(fluo_ch)+' \n')
    f.close()
    
    
    os.chmod(filename, stat.S_IRWXU)
    
    subprocess.call(['sbatch',filename])